<a href="https://colab.research.google.com/github/patelami3431/Web-Crawler/blob/master/HLT_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [125]:
import urllib
from urllib import request
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import re
from nltk.tokenize import sent_tokenize
from urllib.error import HTTPError

In [126]:
def prepare_soup(url):
    try:
        p = request.urlopen(url)
        if(p.getcode() == 403):
            return False
        data = p.read()
        soup = BeautifulSoup(data, 'html.parser')
        return soup
    except HTTPError as e:
        return False


In [110]:
def print_paras(soup):
    text1 = ""
    for p in soup.select("p"):
        text1 += p.get_text()
    #     print(p)
    text1 = re.sub(r"(?<=\.)(?<=\+)(?<=\\)\(https?:\/\/.+\)"," ", text1)
#     print(text1)
    return text1

In [111]:
def visible(element):
    if element.parent.name in ['style', 'script', '[document]', 'head', 'title']:
        return False
    elif re.match('<!--.*-->', str(element.encode('utf-8'))):
        return False
    return True

In [112]:
def find_links(soup):
    s = []
    for link in soup.find_all('a'):
#             print(link.get('href'))
            s.append(link.get('href'))
    return s



In [113]:
def all_text(soup):
    # kill all script and style elements
    for script in soup(['style', 'script', '[document]', 'head', 'title']):
        script.extract()    # rip it out

    # get text
    text = soup.get_text()

    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = ' '.join(chunk for chunk in chunks if chunk)
    return text

In [130]:
def web_crawler():
    soup = prepare_soup("https://en.wikipedia.org/wiki/List_of_diets")
    s = find_links(soup)
    counter = 0
    final = []
    crawl = True
    i = 0
    while (crawl):
        for link in s:
            if link and ('diet' in link or 'Diet' in link)  and link.startswith('http'):
                final.append(link)
                counter += 1
        if (counter > 15):
            crawl = False
            break
        else:
            soup1 = prepare_soup(final[i])
            while soup1 is False:
                i += 1
                soup1 = prepare_soup(final[i])
                s = find_links(soup1)
                continue
            i += 1
            s = find_links(soup1)                
            

#     print(counter)
    
    file = "file_"
    j = 0
    for url in final:
        soup2 = prepare_soup(url)
        if soup2 is False:
            continue
        text = all_text(soup2)
        filename = file + str(j) + ".txt"
        if text:
            with open("Files/" + filename, "w+") as f:
                f.write(text)
            f.close()
            j += 1
    print(j)  

In [132]:
web_crawler()

19
